# 检测人脸边界并裁剪人脸

In [0]:
!pip install face_recognition
!pip install dlib

In [0]:
# use https://github.com/ageitgey/face_recognition
from PIL import Image
import face_recognition
import os
import json

class FaceExtracter(object):
  def __init__(self, image_path:str, output_path:str, bound_store_path:str, final_size:[tuple, list]):
    # path of image want to extract，note: all images should be placed in sub-dir imgs
    self.image_path = image_path + "/"
    self.output_path = output_path + "/"
    self.bound_store_path = bound_store_path + "/"
    self.final_size = final_size
  
  def extractAllFace(self): # 提取图片集中的所有脸
    image_name_list = os.listdir(self.image_path)
    for image_name in image_name_list:
      # 加载图片 (.jpg, .png, etc) 到numpy矩阵
      image = face_recognition.load_image_file(self.image_path+image_name)
      # 返回包含图像中人脸的边界的列表 eg.[(68, 211, 175, 103), ...]
      face_locations = face_recognition.face_locations(image) # use GPU: model="cnn"
      
      print("found {} face(s) in this {}.".format(len(face_locations), image_name))

      for idx, face_location in enumerate(face_locations):
        top, right, bottom, left = face_location 
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image = pil_image.resize(self.final_size,Image.ANTIALIAS)

        if not os.path.exists(self.output_path):
          os.makedirs(self.output_path)

        pil_image.save(self.output_path + image_name.replace(".", "_%d." % idx))

  def extractOneFace(self):
    image_name_list = os.listdir(self.image_path)
    results = {}
    face_cnt = 0
    message = "| "
    
    for image_name in image_name_list: 
      image = face_recognition.load_image_file(self.image_path+image_name)
      face_locations = face_recognition.face_locations(image)
      
      if len(face_locations) == 0: # 未检测到任何人脸，注意这不代表图中真的没有人脸
        message += image_name + " | "
        print("No face in %s！" % image_name)
      
      else:
        face_cnt += 1
        results[image_name] = face_locations
        print("Detect finish: [%s]" % image_name)

        top, right, bottom, left = face_locations[0]
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image = pil_image.resize(self.final_size, Image.ANTIALIAS)
        
        if not os.path.exists(self.output_path):
          os.makedirs(self.output_path)
      
        pil_image.save(self.output_path + image_name)
        print("saved image: [%s]" % image_name)

    if not os.path.exists(self.bound_store_path):
      os.makedirs(self.bound_store_path)

    with open("%s/face_bound.json" % self.bound_store_path, "w") as f:
      json.dump(results, f)

    print("Write successfully！Total Image: [%d], Total Face: [%d], No face detect in %s" % (len(image_name_list), face_cnt, message))
      
      
if __name__ == '__main__':
  extracter = FaceExtracter('/content/drive/My Drive/face/Original_Img/', "/content/drive/My Drive/face/imgs/", "/content/drive/My Drive/face/Face Bound/", (128, 128))
  # extracter.extractAllFace()
  extracter.extractOneFace()


# 拼接人脸

In [30]:
from PIL import Image
import os
import numpy as np
import json


class FaceSplitJoin(object):
  def __init__(self, bg_im_ph:str, f_im_ph:str, fb_ph:str, op_ph:str):
    self.background_img_path = bg_im_ph + "/" # 背景板图片路径
    self.face_img_path = f_im_ph + "/"   # 生成假脸的路径,两者文件名（id）需一一对应！！！
    
    self.output_path = op_ph + "/"   # 拼接后的图片输出路径
    if not os.path.exists(self.output_path):
      os.makedirs(self.output_path)
    
    self.face_bound_dict = {} # 存储人脸边界
    with open(fb_ph, "r") as f:
      self.face_bound_dict = json.loads(f.read())
    if self.face_bound_dict == {}:
      raise Exception("can't load face bound！")

  def split_join(self):
    image_name_list = os.listdir(self.background_img_path)

    for image_name in image_name_list:
      background_image = Image.open(self.background_img_path+image_name) # 打开背景板
      try:
        face_bound = face_bound_list[image_name][0]
        top, right, bottom, left = face_bound

        fake_face_image = Image.open(self.face_img_path+image_name)
        # 必须调整尺寸，因为神经网络模型输出统一尺寸
        fake_face_image = fake_face_image.resize((right-left, bottom-top), Image.ANTIALIAS)

        # 拼接
        background_image_array = np.array(background_image)
        background_image_array[top:bottom, left:right] = np.array(fake_face_image)
        
        image = Image.fromarray(background_image_array.astype('uint8')).convert('RGB')
        image.save("%s/sj_%s" % (self.output_path, image_name))
        print("Split join [%s] successfully!" % image_name)
      except Exception as e:
        # print("Error:", e)
        print("No key: [%s]" % image_name)
    
    print("[Finish]")


if __name__ == '__main__':
  SJ = FaceSplitJoin("/content/drive/My Drive/face/Original_Img/", "/content/drive/My Drive/face/imgs/",
                     "/content/drive/My Drive/face/Face Bound/face_bound.json", 
                     "/content/drive/My Drive/face/Fake face/")
  SJ.split_join()


Split join [16.jpg] successfully!
Split join [33.jpg] successfully!
Split join [15.png] successfully!
Split join [1.jpg] successfully!
Split join [14.jpg] successfully!
Split join [12.jpg] successfully!
Split join [17.jpg] successfully!
Split join [11.jpg] successfully!
Split join [0.jpeg] successfully!
Split join [10.jpg] successfully!
Split join [13.jpeg] successfully!
Split join [19.jpg] successfully!
Split join [18.jpg] successfully!
Split join [20.jpg] successfully!
Split join [22.jpg] successfully!
Split join [23.jpg] successfully!
Split join [24.jpg] successfully!
Split join [25.jpg] successfully!
Split join [2.png] successfully!
Split join [26.jpeg] successfully!
Split join [21.jpg] successfully!
Split join [27.jpg] successfully!
Split join [30.jpg] successfully!
Split join [34.jpg] successfully!
Split join [29.jpg] successfully!
Split join [3.jpg] successfully!
Split join [35.jpg] successfully!
Split join [36.jpg] successfully!
Split join [32.jpg] successfully!
Split join [31.

# 仅检测人脸边界

In [0]:
from PIL import Image
import face_recognition
import os
import json


class FaceBoundDetector(object):
  def __init__(self, image_path:str, output_path:str):
    self.image_path = image_path + "/"
    self.output_path = output_path + "/"
  
  def detect_face_bound(self):
    image_name_list = os.listdir(self.image_path)
    results = {}
    face_cnt = 0
    message = "| "
    for image_name in image_name_list:
      image = face_recognition.load_image_file(self.image_path+image_name)
      face_locations = face_recognition.face_locations(image)
      
      if len(face_locations) == 0:
        message += image_name + " | "
      else:
        face_cnt += 1
        results[image_name] = face_locations
      
      print("Detect finish: [%s]" % image_name)

    with open("%s/face_bound.json" % self.output_path, "w") as f:
      json.dump(results, f)

    print("Write successfully！Total Image: [%d], Total Face: [%d], No face detect in %s" % (len(image_name_list), face_cnt, message))


# 仅为示例，不要运行这个脚本 
#if __name__=="__main__":
#  detector = FaceBoundDetector("/content/drive/My Drive/face/Original_Img/", "/content/drive/My Drive/face/Face Bound/")
#  detector.detect_face_bound()